In [1]:
import os
import pandas as pd
import numpy as np
import ee
from tqdm import tqdm
from datetime import datetime


ee.Authenticate()
ee.Initialize()

# Path to the merged CSV file
input_csv = r"G:\Public\CYGNSS_clip_csv\Merged_data\cygnss_data_month_12.csv"
output_csv = r"G:\Public\CYGNSS_clip_csv\water_fraction\water_body_fraction_2023_12.csv"

# Function to calculate water body fraction
def calculate_water_fraction(region, date):
    # Load Sentinel-1 data
    sentinel1 = ee.ImageCollection("COPERNICUS/S1_GRD") \
        .filterBounds(region) \
        .filterDate(date, date.advance(1, 'day')) \
        .filter(ee.Filter.eq('instrumentMode', 'IW')) \
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))

    if sentinel1.size().getInfo() == 0:
        return None  # No data for the region and date

    # Sentinel-1 VV band
    sentinel1_vv = sentinel1.mosaic().select('VV')

    # Threshold for water detection (can be adjusted based on the region)
    water_mask = sentinel1_vv.lt(-17)

    # Calculate the fraction of water area
    water_area = water_mask.multiply(ee.Image.pixelArea()).reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=region,
        scale=10,
        maxPixels=1e13
    ).get("VV")

    total_area = region.area().getInfo()

    if water_area.getInfo() is None or total_area == 0:
        return None

    return water_area.getInfo() / total_area

# Read the input CSV
data = pd.read_csv(input_csv)

# List to store results
results = []

# Process each row in the CSV
for _, row in tqdm(data.iterrows(), total=len(data), desc="Processing rows"):
    region_coords = [
        [row['top_left_lon'], row['top_left_lat']],
        [row['top_right_lon'], row['top_right_lat']],
        [row['bottom_right_lon'], row['bottom_right_lat']],
        [row['bottom_left_lon'], row['bottom_left_lat']],
        [row['top_left_lon'], row['top_left_lat']]  # Close the polygon
    ]

    region = ee.Geometry.Polygon([region_coords])
    
    # Ensure the date is formatted for GEE
    raw_date = str(row['date'])  # Convert the integer to a string
    formatted_date = datetime.strptime(raw_date, '%Y-%m-%d').strftime('%Y-%m-%d')
    date = ee.Date(formatted_date)  # Use the formatted date for GEE

    # Calculate water fraction
    water_fraction = calculate_water_fraction(region, date)

    if water_fraction is not None:
        #print(region_coords, water_fraction)
        results.append({
            "water_fraction": water_fraction
        })

# Save the results to a new CSV
results_df = pd.DataFrame(results)
results_df.to_csv(output_csv, index=False)
print(f"Water body fractions saved to {output_csv}")

Processing rows:   0%|                               | 192/1774007 [00:50<130:25:45,  3.78it/s]


EEException: Quota exceeded for quota metric 'Number of read requests' and limit 'Number of read requests per minute per user' of service 'earthengine.googleapis.com' for consumer 'project_number:728610983856'.